# Import python package

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN

from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.mixture import GaussianMixture
from sklearn.cluster import MeanShift

from pycaret.regression import setup, compare_models

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
second_df = pd.read_csv('/Users/tomleung/Downloads/Cardiovascular_Disease_Dataset.csv')
main_df = pd.read_csv('/Users/tomleung/Downloads/heart.csv')

In [ ]:
main_df

In [ ]:
second_df

# Data Pre-Processing

In [ ]:
# patientid is not necassary
second_df = second_df.drop('patientid', axis=1)

In [ ]:
# rename the column of second_df
column_mapping = {
    "gender" : "sex",
    "chestpain" : "cp",
    "restingBP" : "trtbps",
    "serumcholestrol" : "chol",
    "fastingbloodsugar" : "fbs",
    "restingrelectro" : "restecg",
    "maxheartrate" : "thalachh",
    "exerciseangia" : "exng",
    "slope" : "slp",
    "noofmajorvessels" : "caa",
    "thal" : "thall",
    "target" : "output"
}
second_df = second_df.rename(columns=column_mapping)

In [ ]:
second_df.head()

In [ ]:
# Merging the datasets
df = pd.concat([second_df, main_df], axis=0, join='outer', ignore_index=True)

In [ ]:
df

In [ ]:
# Removing outliers
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1

print(f"Original DataFrame shape: {df.shape}")

df = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]

print(f"DataFrame shape after outlier removal: {df.shape}")

In [ ]:
df.head()

In [ ]:
df.shape
# check how many row of data

In [ ]:
df.isnull().sum() 
#check any missing value

In [ ]:
df.duplicated().sum()
# sum of duplicate data

In [ ]:
df.drop_duplicates(keep='first',inplace=True)
# remove duplicate data

In [ ]:
# predict the missing values 
thal_present = df[df['thall'].notnull()]
thal_missing = df[df['thall'].isnull()]

X = thal_present.drop('thall', axis=1)
y = thal_present['thall']

imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=0)

model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
# Replace missing 'thal' values with the predicted values
thal_missing_imputed = imputer.transform(thal_missing.drop('thall', axis=1))

thal_predicted = model.predict(thal_missing_imputed)

thal_predicted_rounded = thal_predicted.round(3)

df.loc[df['thall'].isnull(), 'thall'] = thal_predicted_rounded

In [ ]:
df

In [ ]:
df.describe()
# show statistical data

In [ ]:
df.info()

# Association Rule Mining

## Data Description
age - age in years

sex - sex (1 = male; 0 = female)

cp - chest pain type (1 = typical angina; 2 = atypical angina; 3 = non-anginal pain; 0 = asymptomatic)

trestbps - resting blood pressure (in mm Hg on admission to the hospital)

chol - serum cholestoral in mg/dl

fbs - fasting blood sugar > 120 mg/dl (1 = true; 0 = false)

restecg - resting electrocardiographic results (1 = normal; 2 = having ST-T wave abnormality; 0 = hypertrophy)

thalach - maximum heart rate achieved

exang - exercise induced angina (1 = yes; 0 = no)

oldpeak - ST depression induced by exercise relative to rest

slope - the slope of the peak exercise ST segment (2 = upsloping; 1 = flat; 0 = downsloping)

ca - number of major vessels (0-3) colored by flourosopy

thal - 2 = normal; 1 = fixed defect; 3 = reversable defect

num - the predicted attribute - diagnosis of heart disease (angiographic disease status) (Value 0 = < diameter narrowing; Value 1 = > 50% diameter narrowing)

In [ ]:
df_arm = df.copy()
# Binning age into categories
df_arm['age'] = pd.cut(df['age'], bins=[0, 20, 40, 60, 100], labels=['0_To_20', '21_To_40','41_To_60', '60_above'])
df_arm['sex'] = pd.cut(df['sex'], bins=[-1, 0, 1], labels=['Female', 'Male'])
df_arm['cp'] = pd.cut(df['cp'], bins=[-1, 0, 1, 2, 3], labels=['asymptomatic', 'typical_angina', 'atypical_angina', 'non-anginal_pain'])
df_arm['thall'] = pd.cut(df['thall'], bins=[0, 1, 2, 3], labels=['fixed_defect', 'thal_normal', 'reversable_defect'])
df_arm['fbs'] = pd.cut(df['fbs'], bins=[-1, 0, 1], labels=['fasting_blood_sugar<=120 mg/dl', 'fasting_blood_sugar>120_mg/dl'])
df_arm['restecg'] = pd.cut(df['restecg'], bins=[-1, 0, 1, 2], labels=['hypertrophy', 'normal', 'having_ST-T_wave_abnormality'])
df_arm['exng'] = pd.cut(df['exng'], bins=[-1, 0, 1], labels=['not_exercise_induced_angina', 'exercise_induced_angina'])

df_arm['trtbps'] = pd.cut(df['trtbps'], 3, labels=['low_bp', 'medium_bp', 'high_bp'])
df_arm['chol'] = pd.cut(df['chol'], 3, labels=['low_chol', 'medium_chol', 'high_chol'])
df_arm['thalachh'] = pd.cut(df['thalachh'], 3, labels=['low_hr', 'medium_hr', 'high_hr'])
df_arm['oldpeak'] = pd.cut(df['oldpeak'], 3, labels=['low_op', 'medium_op', 'high_op'])
df_arm['output'] = pd.cut(df['output'], bins=[-1, 0, 1], labels=['more_chance_of_heart_attack', 'less_chance_of_heart_attack'])
df_arm

In [ ]:
# rename all of value name, make the value clearly, allow easy to understand
def rename_values(df):
    for col in df.columns:
        if col not in ['Unnamed', 'thall']:
            df[col] = df[col].apply(lambda x: f"{col}_{str(x)}" if pd.notnull(x) else x)
    return df

data_categorical = rename_values(df_arm)
data_categorical.head()

In [ ]:
data_asso_rule = data_categorical.copy()
data_asso_rule = data_asso_rule.astype(str)
data_asso_rule = data_asso_rule.values.tolist()
# Create TransactionEncoder object
te = TransactionEncoder()

# Perform One-hot encoding
te_ary = te.fit_transform(data_asso_rule)

# Convert the sparse matrix to a dense array
data6 = pd.DataFrame(te_ary, columns=te.columns_)

frequent_itemsets = apriori(data6, min_support=0.3, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.4)

filtered_rules = rules[rules['consequents'].astype(str).str.contains("|".join(['output_more_chance_of_heart_attack', 'output_less_chance_of_heart_attack']))]

bundles = []
for index, row in filtered_rules.iterrows():
    antecedents = ', '.join(list(row['antecedents']))
    consequents = ', '.join(list(row['consequents']))
    support = row['support']
    confidence = row['confidence']
    lift = row['lift']
    bundle = f"{antecedents} --> {consequents} [Support: {support:.3f}, Confidence: {confidence:.3f}, Lift: {lift:.3f}]"
    bundles.append(bundle)

# Print each bundle
for bundle in bundles:
    print(bundle)

## Data Visualization

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

sns.histplot(df['age'], bins=20, kde=True, ax=axes[0, 0])
axes[0, 0].set_title('Age Distribution')

sns.histplot(df['trtbps'], bins=20, kde=True, ax=axes[0, 1])
axes[0, 1].set_title('Resting Blood Pressure Distribution')

sns.histplot(df['chol'], bins=20, kde=True, ax=axes[1, 0])
axes[1, 0].set_title('Cholesterol Distribution')

sns.histplot(df['thalachh'], bins=20, kde=True, ax=axes[1, 1])
axes[1, 1].set_title('Max Heart Rate Distribution')

plt.tight_layout()
plt.show()

In [ ]:
# categorical variables
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

sns.countplot(x='sex', data=df, ax=axes[0, 0])
axes[0, 0].set_title('Gender Distribution')
axes[0, 0].set_xticklabels(['Female', 'Male'])

sns.countplot(x='cp', data=df, ax=axes[0, 1])
axes[0, 1].set_title('Chest Pain Type Distribution')

sns.countplot(x='fbs', data=df, ax=axes[1, 0])
axes[1, 0].set_title('Fasting Blood Sugar > 120 mg/dl Distribution')
axes[1, 0].set_xticklabels(['False', 'True'])

sns.countplot(x='exng', data=df, ax=axes[1, 1])
axes[1, 1].set_title('Exercise Induced Angina Distribution')
axes[1, 1].set_xticklabels(['No', 'Yes'])

plt.tight_layout()
plt.show()


In [ ]:
# Creating a heatmap to visualize the correlation between variables
plt.figure(figsize=(9, 7))
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=.5)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
# show data distributions
fig, axes = plt.subplots(2, 2, figsize=(9, 7))

sns.boxplot(x=df['age'], ax=axes[0, 0])
axes[0, 0].set_title('Box Plot of Age')

sns.boxplot(x=df['trtbps'], ax=axes[0, 1])
axes[0, 1].set_title('Box Plot of Resting Blood Pressure (trtbps)')

sns.boxplot(x=df['chol'], ax=axes[1, 0])
axes[1, 0].set_title('Box Plot of Cholesterol (chol)')

sns.boxplot(x=df['thalachh'], ax=axes[1, 1])
axes[1, 1].set_title('Box Plot of Max Heart Rate (thalachh)')

plt.tight_layout()
plt.show()

# Data Processing

## Data Spliting

In [ ]:
X = df.drop('output',axis='columns')
y = df['output']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=0)
X_test = main_df.drop('output',axis='columns')
y_test = main_df['output']

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# from imblearn.over_sampling import SMOTE
# smote = SMOTE(random_state=0)
# X_train_pca, y_train = smote.fit_resample(X_train_pca, y_train)

In [ ]:
# Reduce data dimensions using PCA - reduce 2 dimensions
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

In [ ]:
# Implement K-Means with different number of clusters
scores = {}
for k in range(2, 10):
    kmeans = KMeans(n_clusters=k, random_state=0)
    cluster_labels = kmeans.fit_predict(X_train_pca)
    silhouette_avg = silhouette_score(X_train_pca, cluster_labels)
    scores[k] = silhouette_avg

# Finding the optimal number of clusters
optimal_k = max(scores, key=scores.get)
optimal_score = scores[optimal_k]

In [ ]:
# Agglomerative Clustering and DBSCAN, then use other clustering models
models = {
    "AgglomerativeClustering": AgglomerativeClustering(n_clusters=3),
    "DBSCAN": DBSCAN(eps=0.5, min_samples=5),
    "MeanShift" : MeanShift(bandwidth=0.5), 
    "GaussianMixture": GaussianMixture(n_components=5)
}

for model_name, model in models.items():
    cluster_labels = model.fit_predict(X_train_pca)
    
    silhouette_avg = silhouette_score(X_train_pca, cluster_labels)
    
    print(f'{model_name} Silhouette Scores is {silhouette_avg}')
print(f'K-Means clusting: Optimal k equal {optimal_k} with Silhouette Scores is {optimal_score}')

In [ ]:
label_range = {2: 0.30, 3: 0.45, 4: 0.55, 5: 0.35, 6: 0.25, 7: 0.20, 8: 0.15, 9: 0.10}

optimal_k_example = max(label_range, key=label_range.get)

kmeans = KMeans(n_clusters=optimal_k, random_state=0)
cluster_labels = kmeans.fit_predict(X_test_pca)

plt.figure(figsize=(10, 6))
plt.scatter(X_test_pca[:, 0], X_test_pca[:, 1], c=cluster_labels, cmap='viridis', marker='o')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s=300, c='red', marker='X', label='Centroids')
plt.title(f'Cluster Visualization')
plt.legend()
plt.show()

# Model training

In [ ]:
kmeans = KMeans(n_clusters=optimal_k, n_init=10, random_state=0)
X_train['cluster_label'] = kmeans.fit_predict(X_train_pca)
X_test['cluster_label'] = kmeans.predict(X_test_pca)

x_test = X_test
x_train = X_train

In [ ]:
_ = setup(data=pd.concat([X_train, y_train], axis=1), target='output')

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(x_train, y_train)
  
predicted = model.predict(x_test)
LogisticRegression_score = accuracy_score(y_test, predicted) * 100.0
print("Logistic Regression accuracy: %.2f%%" % LogisticRegression_score)

In [ ]:
from sklearn.svm import SVC

model = SVC()
model.fit(x_train, y_train)
  
predicted = model.predict(x_test)
SVC_score = accuracy_score(y_test, predicted) * 100.0
print("SVC accuracy: %.2f%%" % SVC_score)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier()
model.fit(x_train, y_train)
  
predicted = model.predict(x_test)
GradientBoostingClassifier_score = accuracy_score(y_test, predicted) * 100.0
print("GradientBoosting Classifier accuracy: %.2f%%" % GradientBoostingClassifier_score)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()

model.fit(x_train, y_train)

predicted = model.predict(x_test)
RandomForestClassifier_score = accuracy_score(y_test, predicted) * 100.0
print("Random Forest accuracy: %.2f%%" % RandomForestClassifier_score)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()

model.fit(x_train, y_train)

predicted = model.predict(x_test)
DecisionTreeClassifier_score = accuracy_score(y_test, predicted) * 100.0
print("Decision Tree Classifier accuracy: %.2f%%" % DecisionTreeClassifier_score)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=11, p=2)

model.fit(x_train, y_train)

predicted = model.predict(x_test)
KNeighborsClassifier_score = accuracy_score(y_test, predicted) * 100.0
print("KNeighbors Classifier accuracy: %.2f%%" % KNeighborsClassifier_score)

In [ ]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()

model.fit(x_train, y_train)

predicted = model.predict(x_test)
GaussianNB_score = accuracy_score(y_test, predicted) * 100.0
print("GaussianNB accuracy: %.2f%%" % GaussianNB_score)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier()

model.fit(x_train, y_train)

predicted = model.predict(x_test)
AdaBoostClassifier_score = accuracy_score(y_test, predicted) * 100.0
print("AdaBoost Classifier accuracy: %.2f%%" % AdaBoostClassifier_score)

In [ ]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier()

model.fit(x_train, y_train)

predicted = model.predict(x_test)
MLPClassifier_score = accuracy_score(y_test, predicted) * 100.0
print("MLP Classifier accuracy: %.2f%%" % MLPClassifier_score)

In [ ]:
import xgboost as xgb

model = xgb.XGBClassifier(objective='binary:logistic', random_state=2, learning_rate=0.4)

model.fit(x_train, y_train)

predicted = model.predict(x_test)
xgb_score = accuracy_score(y_test, predicted) * 100.0
print("XGB Accuracy: %.2f%%" % xgb_score)

In [ ]:
from sklearn import tree
# remark: maybe accruacy score seems not a good apporach to evaluate the accuracy of decision tree
model = tree.DecisionTreeClassifier(random_state=0)
model.fit(x_train, y_train)

predicted = model.predict(x_test)
dtc_score = accuracy_score(y_test, predicted) * 100.0
print("Decision TreeC Classifier Accuracy: %.2f%%" % dtc_score)

In [ ]:
# decision tree visuliation
plt.figure(figsize=(30, 16))
tree.plot_tree(model, filled=True)
plt.title("Decision Tree")
plt.show()

In [ ]:
import catboost
from catboost import CatBoostClassifier

cat_clf = CatBoostClassifier()
cat_clf.fit(X_train, y_train)

predicted = cat_clf.predict(x_test)
cat_score = accuracy_score(y_test, predicted) * 100.0
print("catboost Accuracy: %.2f%%" % cat_score)

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import r2_score

model = ExtraTreesRegressor(random_state=0)
model.fit(X_train, y_train)
predicted = model.predict(x_test)
etr_score = r2_score(y_test, predicted) * 100
print("Extra Trees Regressor Accuracy: %.2f%%" % etr_score)

# Evaluation

In [ ]:
models = ['Logistic Regression', 'Gradient Boosting', 'SVC', 'Random Forest Classifier', 
          'Decision Tree Classifier', 'KNeighbors Classifier', 'GaussianNB', 'AdaBoostClassifier',
         'MLPClassifier', 'XGB', 'DecisionTreeClassifier', 'CatBoostClassifier']

scores = [LogisticRegression_score, GradientBoostingClassifier_score, SVC_score, 
          RandomForestClassifier_score, DecisionTreeClassifier_score, KNeighborsClassifier_score,
         GaussianNB_score, AdaBoostClassifier_score, MLPClassifier_score, xgb_score, dtc_score, cat_score]

plt.figure(figsize=(14, 8))  
plt.barh(models, scores)  
plt.ylabel('Model')
plt.xlabel('Accuracy (%)')
plt.title('Model Accuracy Comparison')
plt.xlim([0, 100])  
for i in range(len(scores)):
    plt.text(scores[i], i, f'{scores[i]:.2f}%', va='center')  

plt.show()

In [ ]:
xgb_clf = xgb.XGBClassifier(random_state=0)
rf_clf = RandomForestClassifier(random_state=0)
gb_clf = GradientBoostingClassifier(learning_rate=0.5, max_depth=10, max_features=0.7500000000000001, min_samples_leaf=14, min_samples_split=14, n_estimators=100, subsample=0.9500000000000001)
#cat_clf = CatBoostClassifier()

xgb_clf.fit(x_train, y_train)
rf_clf.fit(x_train, y_train)
gb_clf.fit(x_train, y_train)
#cat_clf.fit(X_train, y_train)

xgb_pred = xgb_clf.predict(X_test)
rf_pred = rf_clf.predict(X_test)
gb_pred = gb_clf.predict(X_test)
cat_clf_pred = cat_clf.predict(X_test)

combined_pred = []
for i in range(len(X_test)):
    votes = [xgb_pred[i], rf_pred[i], gb_pred[i], cat_clf_pred[i]]
    combined_pred.append(max(set(votes), key=votes.count))

combined_accuracy = accuracy_score(y_test, combined_pred) * 100
#print('Combined Model Accuracy: %.2f'% combined_accuracy)
print(f'Combined Model Accuracy: {combined_accuracy}%')

In [ ]:
compare_models()

<!-- from tpot import TPOTClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

digits = load_digits()

tpot = TPOTClassifier(generations=5, population_size=50, verbosity=2)
tpot.fit(x_train, y_train)
print("TPOT Accuracy:", tpot.score(x_train, y_train)) -->